<a href="https://colab.research.google.com/github/gomesluiz/pln-na-pratica/blob/main/01-aquisicao-dados-raspagem-de-dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aquisição de dados com raspagem de dados**

In [ ]:
# Instalação de pacotes para raspagem e análise de dados.
!pip install beautifulsoup4 pandas requests

In [ ]:
# Declara define funções utilitárias utilizadas no notebook.
import datetime
def formata_msg(nivel, msg, componente=None):
    """
    Formata uma mensagem de log incluindo o nível de severidade, timestamp,
    componente (opcional) e a mensagem.

    Parâmetros:
    - nivel (str): Nível de severidade da mensagem (ex: 'INFO', 'ERROR', 'WARNING').
    - msg (str): A mensagem de log propriamente dita.
    - componente (str, opcional): O componente ou módulo do sistema que gera a mensagem.

    Retorna:
    - str: A mensagem de log formatada.
    """
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    if componente:
        return f"[{nivel}] {timestamp} - {componente}: {msg}"
    else:
        return f"[{nivel}] {timestamp} - {msg}"



In [ ]:
# Importa módulos essenciais para funcionalidades do notebook.
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(formata_msg("INFO", "Bibliotecas importadas com sucesso; ambiente pronto."))

In [ ]:
# Define o diretório local para armazenar as bases de dados públicas coletadas.
corpora_caminho = "./corpora"

# Verifica se o diretório especificado já existe no ambiente do Colab.
if not os.path.exists(corpora_caminho):
  # Caso o diretório não exista, cria um novo diretório com o nome 'corpora'.
  # Isso é útil para organizar os arquivos de dados baixados ou gerados.
  os.mkdir(corpora_caminho)

print(formata_msg("INFO", f"Diretório {corpora_caminho} foi criado com sucesso."))

## 1. G1

## Raspagem de dados de notícias do G1

O g1 é um portal de notícias brasileiro mantido pelo Grupo Globo e sob orientação da Central Globo de Jornalismo. Foi lançado em 18 de setembro de 2006, ano que a TV Globo fez 41 anos. O portal disponibiliza o conteúdo de jornalismo das diversas empresas do Grupo Globo - TV Globo, GloboNews, rádio CBN, Jornais O Globo, Extra e Valor Econômico, revista e Globo Rural, entre outras - além de reportagens próprias em formato de texto, fotos, áudio e vídeo. Além das cinco redações próprias situadas no Rio de Janeiro, em São Paulo, Brasília, Belo Horizonte e Recife, afiliadas da TV Globo, jornais, revistas, rádios e as agências de Notícias como Agência Estado, Agência France Presse, Associated Press, EFE, New York Times, Lusa, Reuters e Valor Econômico alimentam o portal de notícias, que é atualizado 24 horas por dia.

In [ ]:
# Declara a URL do site para a raspagem de dados.
url = "https://g1.globo.com/"

In [ ]:
# Define o nome do arquivo onde os dados serão salvos.
noticias_arquivo = "g1_noticias.csv"

# Usa 'os.path.join' para criar um caminho completo, garantindo compatibilidade
# entre sistemas operacionais.
noticias_caminho = os.path.join(corpora_caminho, noticias_arquivo)

# Inicializa uma lista para armazenar os dados das notícias.
noticias_g1 = []

# Faz uma solicitação HTTP GET para a URL especificada.
resposta = requests.get(url)

# Verifica se a solicitação foi bem-sucedida (código 200).
if resposta.status_code == 200:
  # Processa o conteúdo da página se a solicitação for bem-sucedida.
  pagina = resposta.content
  soup = BeautifulSoup(pagina, 'html.parser')

  # Encontra todos os elementos que correspondem ao corpo da notícia.
  noticias = soup.findAll('div', attrs={'class': 'feed-post-body'})

  # Itera sobre cada notícia encontrada.
  for noticia in noticias:
    # Extrai o título da notícia.
    titulo = noticia.find('a', attrs={'class': 'feed-post-link'})

    # Tenta encontrar o subtítulo da notícia.
    subtitulo = noticia.find('div', attrs={'class': 'feed-post-body-resumo'})

    # Adiciona o título, subtítulo (se houver) e o link à lista de notícias.
    if (subtitulo):
      noticias_g1.append([titulo.text, subtitulo.text, titulo['href']])
    else:
      noticias_g1.append([titulo.text, '', titulo['href']])

  # Converte a lista de notícias em um DataFrame para facilitar a manipulação.
  noticias_g1_df = pd.DataFrame(noticias_g1, columns=['Título', 'Subtítulo', 'Link'])

  # Salva os dados coletados no arquivo CSV especificado.
  noticias_g1_df.to_csv(noticias_caminho, index=False)

  print(formata_msg("INFO", f"Dados coletados com sucesso da URL: {url}."))
  print(formata_msg("INFO", f"Os resultados foram salvos no arquivo: {noticias_arquivo}."))
else:
  print("Falha na solicitação, código de status:", resposta.status_code)



In [ ]:
# Carrega os dados do arquivo CSV especificado no DataFrame 'noticias_g1_df'.
noticias_g1_df = pd.read_csv(noticias_caminho)
noticias_g1_df.head(10)